In [1]:
import requests
import json
import pandas as pd

In [2]:
#!pip install folium

In [3]:
import folium
from folium import plugins

# reference: 
#### https://sammi-baba.hatenablog.com/entry/2018/12/25/074017  
#### http://www.ie110704.net/2016/11/11/e-stat-api%E3%82%92python%E3%81%8B%E3%82%89%E4%BD%BF%E3%81%A3%E3%81%A6%E5%9C%B0%E7%90%86%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F/
#### https://medium.com/@rode89/pandas%E8%A6%9A%E6%9B%B8-52252df4c231

In [4]:
with open("api_key.json", "r") as f:
    appid = f.read()

In [5]:
appid = appid.replace("\n", "")

In [6]:
#appid = "###################################"
api_version = "2.1"
base_url = "http://api.e-stat.go.jp/rest/{api_version}/app/json/".format(api_version=api_version)



#get_type="getSimpleStatsData"
get_type="getStatsData"
#get_type="refDataset"
stats_data_id="0003215844"
#cd_cat_01="0000" # 国籍->全て
#cd_cat_02="0000" # 性別->男女
#cd_cat_03="00710" # 集計地域->全域
lv_area="2" # 集計レベル->都道府県レベル
section_header_flg="2" # セクションヘッダー->無し



url = base_url + "{get_type}?\
appId={appid}&\
statsDataId={stats_data_id}&\
lvArea={lv_area}&\
sectionHeaderFlg={section_header_flg}".format(
        get_type=get_type,
        appid=appid,
        stats_data_id=stats_data_id,
        #cd_cat_01=cd_cat_01,
        #cd_cat_02=cd_cat_02,
        #cd_cat_03=cd_cat_03,
        lv_area=lv_area,
        section_header_flg=section_header_flg
        )

# cdCat01={cd_cat_01}&cdCat02={cd_cat_02}&cdCat03={cd_cat_03}&\

In [7]:
#url = "http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?appId={}&lang=J&statsDataId=0003215844&metaGetFlg=N&cntGetFlg=N&sectionHeaderFlg=2".format(api_key)
res = requests.get(url)

In [8]:
res

<Response [200]>

In [9]:
res = res.json()

In [10]:
#res2 = json.loads(res.text)
#assert res1 == res2

In [11]:
def simple_parser(dic, tag):
    res = []
    for k,v in dic.items():
        if k == tag:
            res.append(v)
            return res
        elif isinstance(v, dict):
            for r in simple_parser(v, tag):
                res.append(r)
        else:
            []
    return res

In [12]:
values = simple_parser(res, "VALUE")[0] # list of dict_type objects

In [13]:
values[:2]

[{'@tab': '001',
  '@cat01': '000',
  '@cat02': '001',
  '@area': '01000',
  '@time': '2005000000',
  '@unit': '千人',
  '$': '5628'},
 {'@tab': '001',
  '@cat01': '000',
  '@cat02': '001',
  '@area': '01000',
  '@time': '2010000000',
  '@unit': '千人',
  '$': '5506'}]

In [14]:
class_obj = simple_parser(res, "CLASS_OBJ")[0]

In [15]:
class_obj

[{'@id': 'tab',
  '@name': '表章項目',
  'CLASS': {'@code': '001', '@name': '人口', '@level': '', '@unit': '千人'}},
 {'@id': 'cat01',
  '@name': '男女別',
  'CLASS': [{'@code': '000', '@name': '男女計', '@level': '1'},
   {'@code': '001', '@name': '男', '@level': '1'},
   {'@code': '002', '@name': '女', '@level': '1'}]},
 {'@id': 'cat02',
  '@name': '人口',
  'CLASS': [{'@code': '001', '@name': '総人口', '@level': '1'},
   {'@code': '002', '@name': '日本人人口', '@level': '1'}]},
 {'@id': 'area',
  '@name': '全国・都道府県',
  'CLASS': [{'@code': '01000', '@name': '北海道', '@level': '2'},
   {'@code': '02000', '@name': '青森県', '@level': '2'},
   {'@code': '03000', '@name': '岩手県', '@level': '2'},
   {'@code': '04000', '@name': '宮城県', '@level': '2'},
   {'@code': '05000', '@name': '秋田県', '@level': '2'},
   {'@code': '06000', '@name': '山形県', '@level': '2'},
   {'@code': '07000', '@name': '福島県', '@level': '2'},
   {'@code': '08000', '@name': '茨城県', '@level': '2'},
   {'@code': '09000', '@name': '栃木県', '@level': '2'},
   {'@

In [16]:
cat01 = class_obj[1]["@name"] #column name
cat01

'男女別'

In [17]:
cat02 = class_obj[2]["@name"] #column name
cat02

'人口'

In [18]:
cat03 = class_obj[3]["@name"]
cat03

'全国・都道府県'

In [19]:
cat04 = class_obj[4]["@name"]
cat04

'時間軸（年）'

In [20]:
col_info = {"@cat01":cat01, "@cat02":cat02, "@area":cat03, "@time":cat04 ,"$":"千人"}

cat01_map = {d["@code"]:d["@name"] for d in class_obj[1]["CLASS"]}
cat02_map = {d["@code"]:d["@name"] for d in class_obj[2]["CLASS"]}
cat03_map = {d["@code"]:d["@name"] for d in class_obj[3]["CLASS"]}
cat04_map = {d["@code"]:d["@name"] for d in class_obj[4]["CLASS"]}

df = pd.DataFrame(values, columns=["@cat01", "@cat02", "@area", "@time", "$"]).rename(columns=col_info)

df[cat01] = df[cat01].map(cat01_map)
df[cat02] = df[cat02].map(cat02_map)
df[cat03] = df[cat03].map(cat03_map)
df[cat04] = df[cat04].map(cat04_map)

In [21]:
df = df[(df["男女別"]=="男女計")&(df["人口"]=="総人口")]

In [22]:
df = df[["時間軸（年）", "全国・都道府県", "千人"]]

In [23]:
df["千人"] = df["千人"].apply(lambda x: int(x)*1000)

In [24]:
df.rename(columns={"千人":"人口", "時間軸（年）":"年", "全国・都道府県":"都道府県"}, inplace=True)

In [25]:
df.head(20)

,年,都道府県,人口
0,平成17年,北海道,5628000
1,平成22年,北海道,5506000
2,平成27年,北海道,5382000
3,平成28年,北海道,5352000
4,平成29年,北海道,5320000
5,平成17年,青森県,1437000
6,平成22年,青森県,1373000
7,平成27年,青森県,1308000
8,平成28年,青森県,1293000
9,平成29年,青森県,1278000


In [26]:
df.shape

(235, 3)

# Read geocode data

In [27]:
gdf = pd.read_csv("../prefecturalCapital.csv")
gdf["nam_ja"] = gdf["nam_ja"].apply(lambda x: x + "県" if x not in ("北海道", "東京", "大阪", "京都") else x)

In [28]:
gdf["nam_ja"] = gdf["nam_ja"].apply(lambda x: x + "府" if x in ("大阪", "京都") else x)
gdf["nam_ja"] = gdf["nam_ja"].apply(lambda x: x + "都" if x == "東京" else x)
gdf.rename(columns={"nam_ja":"都道府県"}, inplace=True)
gdf.drop("id", axis=1, inplace=True)

In [29]:
df = pd.merge(df, gdf, on="都道府県", how='left')

In [30]:
df.head()

,年,都道府県,人口,lat,lon
0,平成17年,北海道,5628000,43.064359,141.347449
1,平成22年,北海道,5506000,43.064359,141.347449
2,平成27年,北海道,5382000,43.064359,141.347449
3,平成28年,北海道,5352000,43.064359,141.347449
4,平成29年,北海道,5320000,43.064359,141.347449


# Geomapping

In [31]:
time_series = ["17","22","27","28","29"]

In [36]:
#heat_data = [[row['lat'],row['lon'], row["人口"]] for index, row in df.iterrows()]
heat_data = [[[row['lat'],row['lon'], row["人口"]] for index, row in df[df["年"]=="平成"+year+"年"].iterrows()] for year in time_series]

In [38]:
heat_data

[[[43.064359, 141.347449, 5628000],
  [40.824294, 140.74005400000001, 1437000],
  [39.70353, 141.15266699999998, 1385000],
  [38.268737, 140.872183, 2360000],
  [39.718175, 140.10335600000002, 1146000],
  [38.240127, 140.362533, 1216000],
  [37.750146, 140.466754, 2091000],
  [36.341817, 140.446796, 2975000],
  [36.56575, 139.883526, 2017000],
  [36.391205, 139.060917, 2024000],
  [35.857771, 139.647804, 7054000],
  [35.604563, 140.123179, 6056000],
  [35.689184999999995, 139.691648, 12577000],
  [35.447505, 139.642347, 8792000],
  [37.901699, 139.022728, 2431000],
  [36.695274, 137.211302, 1112000],
  [36.594729, 136.62555, 1174000],
  [36.065220000000004, 136.221641, 822000],
  [35.665102000000005, 138.568985, 885000],
  [36.651282, 138.180972, 2196000],
  [35.39116, 136.722204, 2107000],
  [34.976987, 138.383057, 3792000],
  [35.180246999999994, 136.906698, 7255000],
  [34.730546999999994, 136.50861, 1867000],
  [35.004532, 135.868588, 1380000],
  [35.020996200000006, 135.7531135, 2

In [40]:
japan_map = folium.Map(location=[35, 135], zoom_start=6)
hm = plugins.HeatMapWithTime(heat_data,index=["平成"+year+"年" for year in time_series],auto_play=False,radius=40,max_opacity=1,gradient={0.1: 'blue', 0.25: 'lime', 0.5:'yellow',0.75: 'red'})
hm.add_to(japan_map)
japan_map